1 extract frames form video

In [ ]:
import cv2
import numpy as np
import os

SEQUENCE_LENGTH = 30  # number of frames per sequence
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64  # resize frames

def extract_frames_from_video(video_path, seq_length=SEQUENCE_LENGTH):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
        frame = frame / 255.0  # normalize
        frames.append(frame)
    cap.release()
    
    # split into sequences of seq_length frames
    sequences = []
    for i in range(0, len(frames) - seq_length + 1, seq_length):
        sequences.append(np.array(frames[i:i+seq_length]))
    return sequences


load data from directory

In [ ]:
def load_dataset_from_dir(dataset_dir, classes_list):
    X, y = [], []
    for idx, class_name in enumerate(classes_list):
        class_dir = os.path.join(dataset_dir, class_name)
        for file in os.listdir(class_dir):
            if file.endswith('.mp4'):
                video_path = os.path.join(class_dir, file)
                sequences = extract_frames_from_video(video_path)
                X.extend(sequences)
                y.extend([idx]*len(sequences))
    X = np.array(X)
    y = np.array(y)
    return X, y


Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X, y = load_dataset_from_dir('dataset', CLASSES_LIST)
y = to_categorical(y, num_classes=len(CLASSES_LIST))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True)


compile and train LRCN model

In [ ]:
from tensorflow.keras.optimizers import Adam

model = create_LRCN_model()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=8,
    epochs=10
)
model.save('lrcn_model.h5')

real-time testing with opencv

In [ ]:
import time

cap = cv2.VideoCapture(0)  # webcam
seq = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_resized = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
    frame_norm = frame_resized / 255.0
    seq.append(frame_norm)
    
    if len(seq) == SEQUENCE_LENGTH:
        input_seq = np.expand_dims(np.array(seq), axis=0)  # shape (1, seq_len, H, W, 3)
        pred = model.predict(input_seq)
        class_id = np.argmax(pred)
        label = CLASSES_LIST[class_id]
        seq.pop(0)  # slide window

        cv2.putText(frame, label, (50,50), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0,255,0), 2)

    cv2.imshow('Activity Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
